In [1]:
import json
from rich import print as rprint

In [2]:
post = Post.objects.last()
print(post.title)

Weeknotes 2023-12-25


In [3]:
data = post.body.raw_data

In [4]:
gallery = data[0]["value"][-1]

In [5]:
rprint(gallery)

{
    'id': '9e73b5e3-f6e8-4943-af87-f9beaf34e3a1',
    'type': 'gallery',
    'value': {
        'layout': 'default',
        'gallery': [{'id': '9c199723-758b-425c-a661-1f74a204000a', 'type': 'item', 'value': 2133}]
    }
}

In [16]:
isinstance?

Signature: isinstance(obj, class_or_tuple, /)
Docstring:
Return whether an object is an instance of a class or of a subclass thereof.

A tuple, as in ``isinstance(x, (A, B, ...))``, may be given as the target to
check against. This is equivalent to ``isinstance(x, A) or isinstance(x, B)
or ...`` etc.
Type:      builtin_function_or_method

In [43]:
def migrate_value_block(block):
    # rprint("migrate block: ", block)
    return {
        "layout": "default",
        "gallery": block,
    }



def migrate_post_to_gallery_with_layout(post):
    data = post.body.raw_data
    for toplevel in data:
        for block in toplevel["value"]:
            if block["type"] == "gallery":
                if isinstance(block["value"], list):
                    block["value"] = migrate_value_block(block["value"])
#                 else:
#                     rprint("dont migrate this: ", block["value"])
    return list(data)

In [39]:
post = Post.objects.get(slug="weeknotes-2023-12-04")
# post = Post.objects.last()
print(post.title)

Weeknotes 2023-12-04


In [40]:
body_data = migrate_post_to_gallery_with_layout(post)

migrate block: 
[
    {'id': '9ca8588d-fefc-43f1-9640-025d072a9063', 'type': 'item', 'value': 2107},
    {'id': 'c9b97a98-c2fd-492b-9112-641c0c09c067', 'type': 'item', 'value': 2104},
    {'id': 'f77b2137-d670-458c-877f-0f87f505b4bd', 'type': 'item', 'value': 2101},
    {'id': '5f0d76e5-00dd-4fb2-a1ad-76f4e9e79901', 'type': 'item', 'value': 2099},
    {'id': '5b2601e4-89a3-4c04-8f27-470cebc240ab', 'type': 'item', 'value': 2097}
]

In [34]:
body_data = migrate_post_to_gallery_with_layout(post)

dont migrate this: 
{'layout': 'default', 'gallery': [{'id': '9c199723-758b-425c-a661-1f74a204000a', 'type': 'item', 'value': 2133}]}

In [41]:
post.body = json.dumps(body_data)
post.save()

In [44]:
for post in Post.objects.all():
    body_data = migrate_post_to_gallery_with_layout(post)
    post.body = json.dumps(body_data)
    post.save()